In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# zipcode基本資料
url: https://www.unitedstateszipcodes.org

In [4]:
zipcodes = ['98003', '98008', '98019', '98028', '98030', '98119', '98057', '98117', '98074', '98105', '98116', '98144', '98011', '98029', '98133', '98101']
data = []

for zipcode in zipcodes:
    zipcodeList = [zipcode]
    url = 'https://www.unitedstateszipcodes.org/%s/' % zipcode
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}

    res = requests.get(url, headers=headers)
    bs = BeautifulSoup(res.text, 'html.parser')   
    
    table = bs.find_all('table', class_='table table-hover')
    populationDensity = table[0].find_all('tr')[1].find('td').text.replace(',','')
    housingUnits = table[0].find_all('tr')[2].find('td').text.replace(',', '')
    landArea = table[1].find_all('tr')[0].find('td').text
    medianHouseholdIncome = table[1].find_all('tr')[3].find('td').text.replace(',','').replace('$','')
   
    zipcodeList.append(populationDensity)
    zipcodeList.append(housingUnits)
    zipcodeList.append(landArea)
    zipcodeList.append(medianHouseholdIncome)
    
    table = bs.find_all('table', class_ = 'chart-legend table table-striped table-hover table-condensed')
    householdsWithKids = table[3].find_all('tr')[2].find_all('td')[1].text.replace('&percnt', '')
    zipcodeList.append(householdsWithKids)
    
# 數據都在script中 無法直接用bs解析，只能挑出<script>用re挑出數據    
    a = str(bs.find_all('script')[20]).replace('\n','').replace('\t','')
    year1 = int((re.search('1939 Or Earlier","y":[0-9]+',a).group().replace('1939 Or Earlier","y":', '')))
    year2 = int((re.search('1940s","y":[0-9]+',a).group().replace('1940s","y":', '')))
    year3 = int((re.search('1950s","y":[0-9]+',a).group().replace('1950s","y":', '')))
    year4 = int((re.search('1960s","y":[0-9]+',a).group().replace('1960s","y":', '')))
    year5 = int((re.search('1970s","y":[0-9]+',a).group().replace('1970s","y":', '')))
    year6 = int((re.search('1980s","y":[0-9]+',a).group().replace('1980s","y":', '')))
    year7 = int((re.search('1990s","y":[0-9]+',a).group().replace('1990s","y":', '')))
    year8 = int((re.search('2000s","y":[0-9]+',a).group().replace('2000s","y":', '')))
    year9 = int((re.search('2010 Or Later","y":[0-9]+',a).group().replace('2010 Or Later","y":', '')))
    
# 房屋建造年齡分為1970年前和1970年後    
    before1970 = year1 + year2 + year3 + year4 + year5
    after1970 =  year6 + year7 +year8 + year9
    zipcodeList.append(before1970)
    zipcodeList.append(after1970)
    
    name = table[10].find('tr').find('th').text
    amount = table[10].find('tr').find('td').text
    rate = table[10].find('tr').find_all('td')[1].text.replace('&percnt', '')

    zipcodeList.append(rate)
    
    count = 0
    high = 0
    for i in table[11].find_all('tr')[:5]:
        name = i.find('th').text
        amount = i.find('td').text
        rate = i.find_all('td')[1].text.replace('&percnt', '')
        if count < 2 :
            high += float(rate)
        else:
            zipcodeList.append(float(rate))
        if count == 1:
            zipcodeList.append(high)
        count += 1
        
# 數據都在script中 無法直接用bs解析，只能挑出<script>用re挑出數據     
    a = str(bs.find_all('script')[31]).replace('\n','').replace('\t','')
    level1 = int(re.search('< \$25,000","y":[0-9]+',a).group().replace('< $25,000","y":', ''))
    level2 = int(re.search('\$25,000-\$44,999","y":[0-9]+', a).group().replace('$25,000-$44,999","y":', ''))
    level3 = int(re.search('\$45,000-\$59,999","y":[0-9]+', a).group().replace('$45,000-$59,999","y":', ''))
    level4 = int(re.search('\$60,000-\$99,999","y":[0-9]+', a).group().replace('$60,000-$99,999","y":', ''))
    level5 = int(re.search('\$100,000-\$149,999","y":[0-9]+', a).group().replace('$100,000-$149,999","y":', ''))
    level6 = int(re.search('\$150,000-\$199,999","y":[0-9]+', a).group().replace('$150,000-$199,999","y":', ''))
    level7 = int(re.search('\$200,000\+","y":[0-9]+', a).group().replace('$200,000+","y":', ''))
            
                 
                 
    zipcodeList.append(level1)
    zipcodeList.append(level2 + level3)
    zipcodeList.append(level4 + level5)
    zipcodeList.append(level6 + level7)
                 
    total = level1 + level2 + level3 + level4 + level5 + level6 + level7
    rate1 = level1 / total
    rate2 = (level2 + level3) / total
    rate3 = (level4 + level5) / total
    rate4 = (level6 + level7) / total
    zipcodeList.append(rate1)
    zipcodeList.append(rate2) 
    zipcodeList.append(rate3)              
    zipcodeList.append(rate4)              
                 
    data.append(zipcodeList)
df = pd.DataFrame(data,columns=['zipcode', 'population Density', 'housing Units', 'land Area', 'median Household Income', 'households With Kids',
                      ' built before 1970', ' built after 1970','Car', 'High School', "Associate's degree", "Bachelor's degree",
                      "Master's degree", '<25k', '25k-60k', '60k-150k', '150k+','<25krate', '25k-60k rate', '60k-150k rate', '150k+ rate'])

In [5]:
df.head()

,zipcode,population Density,housing Units,land Area,median Household Income,households With Kids,built before 1970,built after 1970,Car,High School,...,Bachelor's degree,Master's degree,<25k,25k-60k,60k-150k,150k+,<25krate,25k-60k rate,60k-150k rate,150k+ rate
0,98003,3800,18436,11.62,45894,33,8998,10274,84.3,68.1,...,15.5,4.2,4565,6176,6037,868,0.258699,0.349994,0.342117,0.049190
1,98008,4437,9552,5.50,92986,32,7244,2104,78.7,38.0,...,37.0,13.0,950,2026,3673,2306,0.106086,0.226242,0.410162,0.257510
2,98019,141,3907,76.33,95025,45,759,3241,91.1,54.0,...,28.0,7.0,392,762,1784,917,0.101686,0.197665,0.462776,0.237873
3,98028,3606,8566,5.66,82448,33,4091,4439,80.0,43.0,...,30.0,12.0,1281,1499,3612,1564,0.161011,0.188411,0.453997,0.196581
4,98030,4741,12739,7.12,51446,40,4607,7248,86.8,66.9,...,17.7,4.7,2736,3557,4138,714,0.245491,0.319157,0.371288,0.064065


In [ ]:
df.to_csv('zipcodeDataUs.csv', index=0)

# zipcode基本資料
url: https://usa.youbianku.com/zh-hant/zipcode-new/98006

In [7]:
# 稍微看一下抓出來的資料是否正確
zipcodes = ['98003', '98008', '98019', '98028', '98030', '98119', '98057', '98117', '98074', '98105', '98116', '98144', '98011', '98029', '98133', '98101']
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
zipcodeData = []

for zipcode in zipcodes:
    zipcodeList = [zipcode]
    url = 'https://usa.youbianku.com/zh-hant/zipcode-new/%s' % zipcode 
    res = requests.get(url,headers=headers)
    bs = BeautifulSoup(res.text, 'html.parser')
    fieldsetLen = len(bs.find_all('fieldset'))

# 所需資料有3種不同的位置，依據<fieldset>的數量可以得知所需資料所在位置
    if fieldsetLen == 22:
        for i in range(11,16):
            for j in bs.find_all('fieldset')[i].find('ul').find_all('li'):

                zipcodeList.append(j.text)
                
    if fieldsetLen == 21:
        for i in range(10,15):
            for j in bs.find_all('fieldset')[i].find('ul').find_all('li'):

                zipcodeList.append(j.text)
    if fieldsetLen == 20:
        for i in range(9,14):
            for j in bs.find_all('fieldset')[i].find('ul').find_all('li'):

                zipcodeList.append(j.text)
    zipcodeData.append(zipcodeList)
                

In [8]:
# 刪除不需要的資料
data = []
for zipcode in zipcodeData[:]:
    del zipcode[2]
    del zipcode[14]
    lis = []
    for i in zipcode:
        num = re.search('[0-9]+,?.?[0-9]*',i).group().replace(',','')
        lis.append(float(num))
    data.append(lis)

In [9]:
df = pd.DataFrame(data,columns=['zipcode','當前人口', '男性人口', '女性人口', '白人人口', '黑人人口', '西班牙裔人口', '亞洲人口', '夏威夷人口', '印度裔人口', '其他人口', '年齡的中位數', '男性年齡中位數', '女性年齡中位數', '每戶人數', '平均房屋價值', '每戶收入'])

In [10]:
df.head()

,zipcode,當前人口,男性人口,女性人口,白人人口,黑人人口,西班牙裔人口,亞洲人口,夏威夷人口,印度裔人口,其他人口,年齡的中位數,男性年齡中位數,女性年齡中位數,每戶人數,平均房屋價值,每戶收入
0,98003.0,48578.0,21343.0,22808.0,26782.0,5993.0,8064.0,6606.0,1783.0,1157.0,5140.0,34.7,32.8,36.5,2.52,269000.0,57185.0
1,98008.0,24721.0,12131.0,12280.0,17966.0,756.0,1865.0,5354.0,121.0,290.0,990.0,41.0,39.7,42.9,2.61,651900.0,104250.0
2,98019.0,11177.0,5400.0,5325.0,10051.0,92.0,668.0,474.0,39.0,193.0,314.0,37.1,36.8,37.4,2.88,469300.0,126250.0
3,98028.0,24061.0,10120.0,10299.0,17162.0,567.0,1434.0,2632.0,128.0,290.0,643.0,39.6,38.5,40.5,2.55,526200.0,105007.0
4,98030.0,36993.0,16832.0,16937.0,20432.0,5044.0,4618.0,6750.0,736.0,937.0,2527.0,32.4,31.1,33.6,2.81,320400.0,60823.0


In [ ]:
df.to_csv('zipcodeData.csv', index=0)

# 交通分數
url: https://www.walkscore.com/score/98006

In [11]:
# 讀取主資料，獲得每筆資料地址
addresses = pd.read_csv('allData.csv')

In [12]:
df = pd.DataFrame(columns=['address', 'walk', 'transit', 'bike'])
for address in addresses['address']:
    url = 'https://www.walkscore.com/score/%s' % address
    res = requests.get(url)
    bs = BeautifulSoup(res.text, 'html.parser')
    
    walk, transit, bike = 0, 0, 0
    for i in bs.find_all('div', class_='clearfix score-div'):
        info = i.find('img')['src'].replace('//pp.walk.sc/badge/', '').replace('.svg','').replace('/score', '')
        if 'walk' in info:
            walk = re.search('[0-9]+', info).group()
           
        if 'transit' in info:
            transit = re.search('[0-9]+', info).group()
                     
        if 'bike' in info:
            bike = re.search('[0-9]+', info).group()
            
    df = df.append({'address':address,'walk':walk,'transit':transit,'bike':bike}, ignore_index=True)

In [17]:
# 不需要bike score
df = df.iloc[:,:3]
df

,address,walk,transit
0,"21729 Se 288th St, Black Diamond, WA 98010",19,0
1,"31019 229th Ave Se, Black Diamond, WA 98010",3,0
2,"29630 232nd Ave Se, Black Diamond, WA 98010",4,0
3,"32330 2nd Ave, Black Diamond, WA 98010",26,0
4,"29620 232nd Ave Se, Black Diamond, WA 98010",4,0


In [15]:
df.to_csv('trafficScore.csv', index=0)

# 台幣美金匯率
url = https://www.cbc.gov.tw/tw/lp-645-1-1-60.html

In [21]:
exchangeRate = []
for i in range(1, 6):
    url = 'https://www.cbc.gov.tw/tw/lp-645-1-%s-60.html' % str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
    res = requests.get(url, headers = headers)
    bs = BeautifulSoup(res.text, 'lxml')
    
    table = bs.find('table', class_ = 'rwd-table').find_all('tr')[1:]
    for j in table:
        date = j.find('td').text
        rate = j.find_all('td')[1].text
        exchangeRate.append([date, rate])
        
df = pd.DataFrame(exchangeRate, columns = ['soldout', 'ExchangeRate'])

In [22]:
df.head()

,soldout,ExchangeRate
0,2021/08/13,27.860
1,2021/08/12,27.819
2,2021/08/11,27.847
3,2021/08/10,27.831
4,2021/08/09,27.826


In [ ]:
df.to_csv('exchangeRate.csv', index=0)